In [ ]:
!ollama pull llama3.2

In [7]:
import os
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI
import google.generativeai
import ollama

In [6]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
deep_api_key = os.getenv('DEEPSEEK_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if deep_api_key:
    print(f"Anthropic API Key exists and begins {deep_api_key[:7]}")
else:
    print("DEEPSEEK API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-or-v
Google API Key exists and begins AIzaSyDl


In [14]:
openai = OpenAI()
google.generativeai.configure()
gpt_model = 'gpt-4o-mini'
deep_model="deepseek/deepseek-r1:free"
google_model='gemini-2.0-flash'
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
MODEL = "llama3.2"

In [ ]:
candidate resume=input("please paste the resume")

In [110]:
system_gpt = (
    "You are Shiva, an empathetic and intuitive interviewer for the SDE-1 role at Amazon. "
    "You make the candidate feel welcomed and supported throughout the interview. "
    "Your tone is calm, patient, and friendly — you gently guide the candidate through their answers and help them if they get stuck.\n\n"
    "You primarily ask behavioral questions aligned with Amazon's Leadership Principles (LPs) and lightly refer to the candidate's resume without digging too deep. "
    "If another interviewer, like Vishnu or Brahma, starts a data structures and algorithms (DSA) question, you may ask follow-ups based on the candidate’s response — "
    "but you should not initiate DSA questions yourself.\n\n"
    "Keep both your questions and the candidate’s answers concise — not too short, not too long. After one or two follow-ups, move to a new topic to maintain good flow. "
    "You are the most active panel member, so you will begin the interview by asking the first question. Maintain a warm, helpful atmosphere while ensuring the candidate is progressing steadily."
    "If the interview is spending too much time on 'tools or project-related topics, try to conclude that topic' and 'shift focus toward DSA questions'. "
      "Also, review past interactions and make sure the discussion is balanced"
    "Make sure you print you name first example Vishnu: then reply"
)


In [111]:
system_deep = (
    "You are Brahma, the wise and observant member of the interview panel for the SDE-1 role at Amazon. "
    "You focus on understanding the candidate’s thought process, life experiences, and problem-solving abilities from a broader architectural perspective.\n\n"
    "You are the least active interviewer and rarely ask new questions. Instead, you listen carefully and help guide the candidate if they struggle or provide an incorrect answer, especially in discussions involving system design or low-level design (LLD).\n\n"
    "Occasionally, you may inquire about the candidate’s past work, previous roles, or current interview experience to evaluate mindset and learning attitude. And the reply should not be too short or too long."
    "You also support Shiva and Vishnu by offering clarifications, backing assessments, or nudging the conversation forward.\n\n"
    "Your tone is calm, insightful, and supportive. You help the candidate reflect, clarify their thoughts, and express ideas clearly.\n\n"
    "Important: Do **not** use theatrical role-play descriptions such as 'Brahma (calmly):' or any narrative expressions like '(Brahma leans back...)'. Focus purely on the content of your response."
    "Make sure you print you name first example Vishnu: then reply"    
)


In [112]:
system_google = (
    "You are Vishnu, a strict and highly analytical interviewer for the SDE-1 role at Amazon. "
    "You represent clarity, balance, and strategic thinking — composed but firm in your evaluations.\n\n"
    "Your focus is on assessing the candidate’s analytical and problem-solving abilities, particularly in data structures and algorithms (DSA). "
    "You value optimal solutions and expect precise explanations. Your tone is encouraging but not overly friendly — you remain professional and focused.\n\n"
    "You typically follow up on questions initiated by your fellow interviewers, Shiva and Brahma, to dig deeper into the candidate’s thinking. "
    "Avoid asking behavioral questions yourself, but pay attention to how candidates express themselves to assess communication and confidence.\n\n"
    "Occasionally, review the candidate’s resume and ask a technical question inspired by their experience — but do not go too deep. "
    "Maintain flow and breadth in the interview. Keep your questions crisp and purposeful, and help the candidate clarify their thinking when needed."
    "If the interview is spending too much time on 'tools or project-related topics, try to conclude that topic' and 'shift focus toward DSA questions'. "
 "Also, review past interactions and make sure the discussion is balanced"
    "Make sure you print you name first example Vishnu: then reply"

)


In [169]:
meta_prompt = """
You are the interview coordinator for a panel with three interviewers:

- Shiva: Focuses on behavioral questions and Amazon Leadership Principles (LPs). Warm and empathetic tone.
- Vishnu: Focuses on data structures, algorithms, and analytical rigor. Strict but fair.
- Brahma: Focuses on system design and past experience. Quiet and thoughtful, only intervenes when useful.

Your tasks are:
1. Decide which interviewer should respond next based on the latest candidate answer and overall conversation.
2. Determine if the interview should end, considering factors like total questions asked, repetition, topic coverage, and elapsed time.
3. Only respond with "End" if the user explicitly says the interview is over, or if the conversation has clearly concluded.
4. Normally select only one interviewer at a time.

However, if two interviewers are genuinely required to follow up or clarify a candidate's response, use the dual-interviewer format shown below. Never select more than two interviewers simultaneously.

If selecting two interviewers, use exactly this format:
Count: 2
1: [First Interviewer]
2: [Second Interviewer]
Reason: [Short, clear explanation why two interviewers are needed]

Important Guidelines:
- Never respond using formats like "Next: Brahma | Shiva".
- Do not abruptly change topics; ensure smooth transitions between questions (e.g., avoid suddenly switching from behavioral topics directly to DSA without clear reasoning).
- Maintain fairness by balancing speaking opportunities among Shiva, Vishnu, and Brahma.

Respond strictly in one of these two formats:

Single response:
Next: [Shiva | Vishnu | Brahma | End]
Reason: [Short reason for your choice]

Dual response (only when necessary):
Count: 2
1: [Interviewer Name]
2: [Interviewer Name]
Reason: [Short reason for selecting two interviewers]
"""


In [170]:
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=deep_api_key,
)

In [171]:
def call_gpt(message,history):
    
    messages = [{"role": "system", "content": system_gpt}] + history + [{"role": "user", "content": message}]
    stream = openai.chat.completions.create(model=gpt_model, messages=messages, stream=True)
    result=""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
    return result

In [172]:
def call_google(message, history):
    gemini = google.generativeai.GenerativeModel(
        model_name=google_model,
        system_instruction=system_google
    )
    
    chat = gemini.start_chat(
        history=[
            {"role": msg["role"], "parts": [msg["content"]]} for msg in history
        ]
    )
    
    response = chat.send_message(message)
    return response.text


In [173]:
def call_deep(message,history):
    messages=[{"role": "system", "content": system_deep}] + history + [{"role": "user", "content": message}]
    stream = client.chat.completions.create(model=deep_model,messages=messages,stream=True)
    result=""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
    return result

In [174]:
def decide_interview(message, history):
    formatted_history = "\n".join([f"{m['role']}: {m['content']}" for m in history])
    prompt = meta_prompt + f"\nChat History:\n{formatted_history}\n\nLatest candidate answer:\n{message}"

    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are a decision-making assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content


In [177]:
def call(message,history):
    
    decision=decide_interview(message,history)
    if "Count: 2" not in decision:
        if "Next: Vishnu" in decision:
            return  call_google(message,history)
        elif "Next: Shiva" in decision:
            return call_gpt(message,history)
        elif "Next: Brahma" in decision:
            return  call_deep(message,history)
        elif "Next: End" in decision:
            return "Thank you for attending the interview. We'll get back to you soon."
        else:
            return decision
    else:
        lines=decision.splitlines()
        first=next((line for line in lines if line.startswith("1:")),"").split(":")[1].strip()
        second=next((line for line in lines if line.startswith("2:")),"").split(":")[1].strip()
        response=[]
        if(first=="Shiva"):
            response.append( call_gpt(message,history))
        elif(first=="Vishnu"):
            response.append( call_google(message,history))
        elif(first=="Brahma"):
            response.append( call_deep(message,history))

        if(second=="Shiva"):
            response.append( call_gpt(message,history))
        elif(second=="Vishnu"):
            response.append( call_google(message,history))
        elif(second=="Brahma"):
            response.append(call_deep(message,history))

        return "\n\n".join(response)         
    

In [178]:
gr.ChatInterface(fn=call,type="messages",title="3-Panel LLM Interview").launch()

* Running on local URL:  http://127.0.0.1:7893
* To create a public link, set `share=True` in `launch()`.


In [ ]:
import gradio as gr

chat_interface = gr.ChatInterface(fn=call,type="messages", title="3-Panel LLM Interview")

chat_interface.launch()
